1 - Importações

In [70]:
# Importar bibliotecas necessárias
import pandas as pd
import os
!pip install -q pandas openpyxl

2 - Definição de constantes

In [71]:
# Define file paths
turma_a_cursistas_file = '/home/emanoel/Downloads/avamec_inscricoes/Cursistas Turma A/Cursistas Turma_A.xlsx'
turma_b_cursistas_file = '/home/emanoel/Downloads/avamec_inscricoes/Cursistas Turma B/Cursistas Turma_B.xlsx'

# Define matching columns
matching_column_cursistas = '[0] login'
matching_column_ava = 'Login do Membro'
error_column_ava = 'Erro Apresentado?'
login_existente_column = 'Login existente'

In [72]:
def verificar_arquivos_existem(turma_a_cursistas_file, emails_file=None):
    """
    Verifica se os arquivos necessários existem.
    
    Args:
        cursistas_file (str): Caminho para o arquivo de cursistas
        emails_file (str, optional): Caminho para o arquivo de emails
    """
    errors = []
    
    if not os.path.exists(turma_a_cursistas_file):
        errors.append(f"Arquivo de cursistas não encontrado: {turma_a_cursistas_file}")
    
    if emails_file and not os.path.exists(emails_file):
        errors.append(f"Arquivo de emails não encontrado: {emails_file}")
        
    return errors

3 - Carregar dados dos cursistas

In [73]:
# Load the cursistas dataframe
try:
    df_cursistas = pd.read_excel(turma_a_cursistas_file)    # Load the cursistas dataframes
    try:
        # Carregar Turma A
        df_cursistas_a = pd.read_excel(turma_a_cursistas_file)
        df_cursistas_a[matching_column_cursistas] = df_cursistas_a[matching_column_cursistas].astype(str).str.strip().str.lower()
        df_cursistas_a['turma'] = 'A'  # Adiciona identificador da turma
        print("Arquivo da Turma A carregado com sucesso!")
        
        # Carregar Turma B
        df_cursistas_b = pd.read_excel(turma_b_cursistas_file)
        df_cursistas_b[matching_column_cursistas] = df_cursistas_b[matching_column_cursistas].astype(str).str.strip().str.lower()
        df_cursistas_b['turma'] = 'B'  # Adiciona identificador da turma
        print("Arquivo da Turma B carregado com sucesso!")
        
        # Combinar as duas turmas em um único DataFrame
        df_cursistas = pd.concat([df_cursistas_a, df_cursistas_b], ignore_index=True)
        print(f"\nTotal de cursistas carregados: {len(df_cursistas)}")
        print(f"Turma A: {len(df_cursistas_a)} cursistas")
        print(f"Turma B: {len(df_cursistas_b)} cursistas")
        
    except FileNotFoundError as e:
        print(f"Error loading cursistas file: {e}. Make sure the file path is correct.")
        raise SystemExit(f"Error loading cursistas file: {e}. Cannot proceed.")
    except Exception as e:
        print(f"Erro inesperado ao carregar arquivos: {e}")
        raise SystemExit(f"Cannot proceed due to unexpected error: {e}")
    # Clean the matching column in df_cursistas
    df_cursistas[matching_column_cursistas] = df_cursistas[matching_column_cursistas].astype(str).str.strip().str.lower()
except FileNotFoundError as e:
    print(f"Error loading cursistas file: {e}. Make sure the file path is correct.")
    raise SystemExit(f"Error loading cursistas file: {e}. Cannot proceed.")

Arquivo da Turma A carregado com sucesso!
Arquivo da Turma B carregado com sucesso!

Total de cursistas carregados: 421
Turma A: 211 cursistas
Turma B: 210 cursistas


In [74]:
# Visualizar primeiras linhas do DataFrame
display(df_cursistas.head())

,[0] login,[1] Nome do usuário,[2] E-mail,[3]CPF,[4] Data de nascimento (opcional),"[5] Senha (opcional - caso não informado, será gerado automaticamente)","[5] Função (Administrador, Professor, Tutor ou Aluno)",Obs: e-mail que entra no meet,turma
0,claudemir-de-oliveira-filho,CLAUDEMIR DE OLIVEIRA FILHO,claudimirdeoliveira2017@gmail.com,80858295172,1973-01-31 00:00:00,8.085830e+10,Aluno,NaN,A
1,claudia-angelica-do-nascimento-alves-santos,Cláudia Angélica do Nascimento Alves Santos,claudiaangel81.ca@gmail.com,3841221408,1981-12-02 00:00:00,3.841221e+09,Aluno,NaN,A
2,claudia-nubia-lira-do-nascimento-lima,CLAUDIA NUBIA LIRA DO NASCIMENTO LIMA,claudianubialira@gmail.com,3496215437,1981-02-28 00:00:00,3.496215e+09,Aluno,NaN,A
3,dulce-cristina-dos-santos-barbosa,Dulce Cristina dos Santos Barbosa,crisdulce2012@gmail.com,34476636500,1963-04-02 00:00:00,3.447664e+10,Aluno,NaN,A
4,edivaldo-da-costa-pontes,Edivaldo da Costa Pontes,edi.historia.pontes@gmail.com,65056280291,1976-12-07 00:00:00,6.505628e+10,Aluno,NaN,A


4 - Função de verificação de emails capturados em uma sessão do meet

In [87]:
def verificar_emails(df_cursistas, emails_file):
    """
    Verifica se os emails fornecidos existem na planilha de cursistas.
    
    Args:
        df_cursistas (pd.DataFrame): DataFrame com dados dos cursistas de ambas as turmas
        emails_file (str): Caminho para o arquivo com lista de emails
    """
    # Verificar existência do arquivo de emails
    if not os.path.exists(emails_file):
        print(f"Erro: Arquivo de emails não encontrado: {emails_file}")
        return
        
    try:
        # Verifica se a coluna de email existe
        if '[2] E-mail' not in df_cursistas.columns:
            print("Erro: Coluna '[2] E-mail' não encontrada na planilha de cursistas")
            return
        
        # Carrega arquivo com emails para verificar
        try:
            with open(emails_file, 'r') as f:
                emails_verificar = [email.strip().lower() for email in f.readlines() if email.strip()]
        except UnicodeDecodeError:
            print("Erro: Arquivo de emails com encoding inválido. Use UTF-8")
            return
            
        if not emails_verificar:
            print("Erro: Arquivo de emails está vazio")
            return
        
        # Converte emails dos cursistas para lowercase para comparação
        emails_cursistas = df_cursistas[['[2] E-mail', 'turma']].copy()
        emails_cursistas['[2] E-mail'] = emails_cursistas['[2] E-mail'].str.strip().str.lower()
        
        # Verifica emails
        emails_encontrados = []
        emails_nao_encontrados = []
        emails_invalidos = []
        turmas_encontradas = []
        
        for email in emails_verificar:
            if '@' not in email:
                emails_invalidos.append(email)
            else:
                # Procura o email e sua turma correspondente
                encontrado = emails_cursistas[emails_cursistas['[2] E-mail'] == email]
                if not encontrado.empty:
                    emails_encontrados.append(email)
                    turmas_encontradas.append(encontrado['turma'].iloc[0])
                else:
                    emails_nao_encontrados.append(email)

        # Exibe apenas os emails não encontrados
        if emails_nao_encontrados:
            print("\nEmails não encontrados nas Turmas A e B:")
            for email in emails_nao_encontrados:
                print(f"{email}")
        else:
            print("\nTodos os emails foram encontrados!")

        # Exibe resultados
        print("\nResultados da verificação de emails:")
        print(f"Total de emails verificados: {len(emails_verificar)}")
        print(f"Emails encontrados: {len(emails_encontrados)}")
        print(f"Emails não encontrados: {len(emails_nao_encontrados)}")
        print(f"Emails inválidos: {len(emails_invalidos)}")
        
        # if emails_nao_encontrados:
        #     print("\nEmails não encontrados na planilha:")
        #     for email in emails_nao_encontrados:
        #         print(f"{email}")  # Removed the hyphen
        
        if emails_invalidos:
            print("\nEmails com formato inválido:")
            for email in emails_invalidos:
                print(f"- {email}")
        
        if emails_encontrados:
            print("\nEmails encontrados (com suas turmas):")
            for email, turma in zip(emails_encontrados, turmas_encontradas):
                print(f"Turma {turma}: {email}")
                
    except pd.errors.EmptyDataError:
        print("Erro: A planilha de cursistas está vazia")
    except Exception as e:
        print(f"Erro inesperado: {e}")

5 - Execução da verificação

In [88]:
# Exemplo de uso da função atualizada
verificar_emails(
    df_cursistas,  # DataFrame unificado com ambas as turmas
    '/home/emanoel/Downloads/avamec_inscricoes/emails_verificar.txt'
)


Emails não encontrados nas Turmas A e B:
anakatia.fn@hotmail.com
adrianamadja@gmail.com
alexsantana7@gmail.com
heidyani@gmail.com
karla.lopes@ufc.br
karlacristiner@ufc.br
marques2022ufc@gmail.com
sumarafrotadn@gmail.com
vitoriaramos@ufc.br
vsalesp4@gmail.com

Resultados da verificação de emails:
Total de emails verificados: 207
Emails encontrados: 197
Emails não encontrados: 10
Emails inválidos: 0

Emails encontrados (com suas turmas):
Turma A: accacia.vicente@gmail.com
Turma A: adjuntosemed025@gmail.com
Turma A: aedyggomes@gmail.com
Turma A: albuquerquenagela@gmail.com
Turma A: alexandrabraga02@gmail.com
Turma A: alexsandracavalcanti531@gmail.com
Turma A: alinenunesje@gmail.com
Turma A: ambrosinasilva2605@gmail.com
Turma A: ana.bacelar@educaita.com.br
Turma A: anamariabarbaralima436@gmail.com
Turma A: andreia05nogueira@gmail.com
Turma A: ang.roarelli@gmail.com
Turma A: aparecidooliveiratrindade93@gmail.com
Turma A: apfqueiroz@yahoo.com.br
Turma A: balbe02.liliane@gmail.com
Turma A: b